# Twirling of effective noise channel

In [1]:
from pyquil.quil import *
from pyquil.api import get_qc
from pyquil.gates import *
from pyquil.latex import display, to_latex
from pyquil.simulation.tools import lifted_gate, program_unitary, lifted_gate_matrix

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from functions import *

## Generation of Haar random circuits

In [8]:
target_qubits = [0,1]
prog = give_random_two_qubit_circuit(target_qubits)
prog.instructions

[<Gate RZ(2.3452969482901067) 0>,
 <Gate RX(pi/2) 0>,
 <Gate RZ(1.1540544441758422) 0>,
 <Gate RX(-pi/2) 0>,
 <Gate RZ(5.353164434037813) 0>,
 <Gate RZ(2.855752157046368) 1>,
 <Gate RX(pi/2) 1>,
 <Gate RZ(1.4746251231135763) 1>,
 <Gate RX(-pi/2) 1>,
 <Gate RZ(0.965262183265261) 1>,
 <Gate CNOT 1 0>,
 <Gate RZ(2.6493205899913583) 0>,
 <Gate RY(1.210610679794906) 1>,
 <Gate CNOT 0 1>,
 <Gate RY(3.2333615161180886) 1>,
 <Gate CNOT 1 0>,
 <Gate RZ(5.583693181667541) 0>,
 <Gate RX(pi/2) 0>,
 <Gate RZ(1.6669675304762166) 0>,
 <Gate RX(-pi/2) 0>,
 <Gate RZ(3.510361248432692) 0>,
 <Gate RZ(5.535982243375302) 1>,
 <Gate RX(pi/2) 1>,
 <Gate RZ(1.6669675304762166) 1>,
 <Gate RX(-pi/2) 1>,
 <Gate RZ(5.628523462631471) 1>]

### Find the (4,4)-matrix representation of each gate

In [12]:
matrix_list = []
for gate in prog.instructions:
    matrix_list.append( program_unitary( Program(gate), n_qubits = 2) )
matrix_list

[array([[0.38771173-0.92178067j, 0.        +0.j        ,
         0.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.38771173+0.92178067j,
         0.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.        +0.j        ,
         0.38771173-0.92178067j, 0.        +0.j        ],
        [0.        +0.j        , 0.        +0.j        ,
         0.        +0.j        , 0.38771173+0.92178067j]]),
 array([[0.70710678+0.j        , 0.        -0.70710678j,
         0.        +0.j        , 0.        +0.j        ],
        [0.        -0.70710678j, 0.70710678+0.j        ,
         0.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.        +0.j        ,
         0.70710678+0.j        , 0.        -0.70710678j],
        [0.        +0.j        , 0.        +0.j        ,
         0.        -0.70710678j, 0.70710678+0.j        ]]),
 array([[0.8380881-0.5455349j, 0.       +0.j       , 0.       +0.j       ,
 